<a href="https://colab.research.google.com/github/kicasta/Modeling_WUGS_WSBM/blob/master/example/example_modeling_wugs_wsbm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Prepare everything

We first need to install every dependency needed to run this notebook in colab. 

In [ ]:
!pip install pymc3 --upgrade

!echo "deb http://downloads.skewed.de/apt bionic main" >> /etc/apt/sources.list
!apt-key adv --keyserver keys.openpgp.org --recv-key 612DEFB798507F25
!apt-get update
!apt-get install python3-graph-tool python3-cairo python3-matplotlib

!apt install libgraphviz-dev
!pip install pygraphviz

!pip install pyvis

We also need to clone the repository to your content, so every module is accessible from this notebook. Notice that your content in colab gets purged everytime your environment is restarted. 

In [ ]:
!git clone https://github.com/kicasta/Modeling_WUGS_WSBM.git

In [ ]:
import sys
sys.path.insert(0,'/content/src/')